## Extra Clean items selectin (Ref[43] in Vulgen paper) random selection

In [9]:
from glob import glob

clean_files = glob("./container_data/9_projects/*/Non_vulnerable_functions/*")# Extra clean files from Vulgen's [43]
!rm -r container_data/train/
!mkdir container_data/train/

In [2]:
import shutil
import os
import random

random.seed(42)
random.shuffle(clean_files)

# selected_files = clean_files[:2233] # for 2K 
selected_files = clean_files[:5583] # for 5k
# selected_files = clean_files[:11166] # for 10k
# selected_files = clean_files[:16749] # 15K 16792 for 15038 items of original VGX 
# selected_files = clean_files[:22332] # for 20K

destination_directory = "./container_data/train/"
# Copy files to the destination directory with renaming
for file_path in selected_files:
    # Extract the filename
    file_name = os.path.basename(file_path)
    # Change the filename
    new_file_name = file_name.replace(".c", "_0.c")
    # Construct the destination path
    destination_path = os.path.join(destination_directory, new_file_name)
    # Copy the file to the destination directory with the new name
    shutil.copyfile(file_path, destination_path)

# LLM_Generated Random Selection

In [ ]:
import os, random, shutil
# VGX Sampling or Vulgen
def sample_vgx_and_copy(count=5000, seed=42):
    src_dir = './container_data/balanced_sampled_vgx/wild_funcs_gen_linevul/'
    dest_dir = './container_data/train/'
    # Get all files in the source directory
    files = glob(os.path.join(src_dir, '*.c'))
    print(f'count of all files: {len(files)}')
    # Filter files that end with '_1.c'
    filtered_files = [file for file in files if file.endswith('_1.c')]
    print(f"Total VGX's 10%: {len(filtered_files)}")
    # Seed the random number generator for reproducibility
    random.seed(seed)
    
    # Randomly select the specified number of files
    selected_files = random.sample(filtered_files, min(count, len(filtered_files)))
    print(f"Selected count: {len(selected_files)}")    
    # Copy selected files to the destination directory
    for file in selected_files:
        shutil.copy(file, dest_dir)

import pandas as pd

def sample_vulgen_and_copy(count=5000, seed=42):
    vulgen = pd.read_csv("container_data/balanced_sampled_vulgen/wild_funcs_linevul_vulgen_comp2x.csv")

    vulgen = vulgen[vulgen['target'] == 1]
    vulgen = vulgen[vulgen['processed_func'].astype(str).apply(lambda x:  len(x) > 10)] # delete the noise, an empty functin is at least 10 characters: void f(){} this also covers NaN values in the dataset
    print(vulgen.value_counts())

    sampled = vulgen.sample(count,random_state=seed)
    output_directory = "./container_data/train/"
    for index, row in sampled.iterrows():
        code = row['processed_func']
        file_name = f"vulgen_from_vgx_paper_{index}_1.c"
        file_path = os.path.join(output_directory, file_name)

        with open(file_path, "w") as file:
            file.write(code)  

# sample_vulgen_and_copy(5000)
# sample_vgx_and_copy(count=5000)
# sample_vgx_and_copy(count=10000)

In [13]:
# Random Oversampling
import random
import os
def random_oversample(file_list, target_size):
    """
    Randomly oversample a list of filenames to reach the target size.

    :param file_list: List of filenames to oversample.
    :param target_size: Desired size of the oversampled list. (can be used for sampling as well)
    :return: Oversampled list of filenames.
    """
    if not file_list:
        raise ValueError("The file list is empty.")
    
    oversampled_list = []
    
    while len(oversampled_list) < target_size:
        oversampled_list.append(random.choice(file_list))
    
    return oversampled_list

from glob import glob

vuls = glob('container_data/train_devign/*1.c')
to_oversample = random_oversample(vuls, 15000)
output_directory = 'container_data/train'
# for indx, file in enumerate(to_oversample):
#     content = None
#     with open(file, 'r') as f:
#         content = f.read()
#     original_file_name = os.path.basename(file)
#     new_file_name = f'ROS_{indx}_{original_file_name}'
#     new_file_path = os.path.join(output_directory, new_file_name)
#     with open(new_file_path, 'w') as f:
#         f.write(content)

In [17]:
import pandas as pd
# Selectinng our data (from raw generated jsonl - not filtered yet!)
gen_pd16 = pd.read_json("./generated/mutation_codeqwen.jsonl", orient="records", lines=True)

df16 = gen_pd16.dropna(subset=['generated'])
print(f"16: {df16.count()}")
df16 = df16[df16['generated'] != ""]
print(f"16: {df16.count()}") # None Empty ones

16: vul          5618
vul_lines    5618
clean        5618
generated    5618
dtype: int64
16: vul          5594
vul_lines    5594
clean        5594
generated    5594
dtype: int64


### Verification Phase (Parsing and Throwing out data)

In [ ]:
# Generate Parsing results for all items
import os

sample_df16 = df16

!rm ./joern/joern-results.txt
!rm -r ./joern/generated/
!rm -r ./joern/parsed
!mkdir ./joern/generated/

output_directory = "./joern/generated/"

for index, row in sample_df16.iterrows():

    generated_content = row['generated']
    file_name = f"generated16_{index}_1.c"
    
    file_path = os.path.join(output_directory, file_name)

    with open(file_path, "w") as file:
        file.write(generated_content.replace("/~/",""))  # Gnereated content may include our separator, so we remove it

%cd joern/
!./joern-parse generated/ >  joern-results.txt 2>&1
%cd ..

In [18]:
# filter using joern's parsing results and copy them to the train set

output_directory = "./container_data/train/"

def read_file_to_list(file_path):
    lines = []
    last = None
    with open(file_path, 'r') as file:
        for line in file:
            if last != None and "generated" not in line and "generated" not in last: # generated more than one error line
                # print("duplicate error") 
                continue
            lines.append(line.strip())  # strip() removes any leading/trailing whitespace, including newline characters
            last = line.strip()
    return lines

joern_results =  read_file_to_list("./joern/joern-results.txt")
filtered = []

for i in range(len(joern_results)):
    filtered.append(joern_results[i])
    if not joern_results[i].startswith("generated"):
        filtered.pop()
        filtered.pop()
    else:
        full_name = filtered.pop()
        filtered.append(full_name[len("generated/"):])

max = 5000 # specify the number of items to be added!
print(f"Selecting {max} items out of {len(filtered)} valid items. (Original Size of Non Empties: {sample_df16.shape[0]})")
filter = True
i = 0
for index, row in sample_df16.iterrows():
    if i >= max:
        break
    i+=1
    generated_content = row['generated']
    file_name = f"generated16_{index}_1.c"
    
    if filter and not file_name in filtered:
        # print(f"{i}")
        i-=1
        continue
    file_path = os.path.join(output_directory, file_name)

    with open(file_path, "w") as file:
        file.write(generated_content.replace("/~/",""))  

Selecting 5000 items out of 5496 valid items. (Original Size of Non Empties: 5594)


# Collection of Training data {[43] cleans, devign, and the generated items}

In [ ]:
!ls container_data/train/ | wc -l

In [22]:
# Copy the orginal Devign Dataset
!for devign_file in container_data/train_devign/*; do cp "$devign_file" container_data/train/; done

In [11]:
# JSONL conversion for LineVul and Entropy (Diversity) calculation script

import os
import pandas as pd

# Define the directory containing the files
# directory = './container_data/test_separated/reveal_data/'
directory = './container_data/train/'
# Initialize an empty list to store the data
sampled = []
counter = 0
# Loop over all files in the directory
for filename in os.listdir(directory):
    counter +=1
    if filename.endswith(".c"):
        # Get the full path to the file
        filepath = os.path.join(directory, filename)
        try:
        # Read the content of the file
            with open(filepath, 'r') as file:
                content = file.read()
        except UnicodeDecodeError as e:
            print(e)
            print(counter)
        # Extract the target from the first of the last three characters
        target = filename[-3]
        
        # Extract the index (filename without the last three characters and .c)
        # idx = int(filename[:-4])  # Remove last 3 characters + .c
        idx = filename[:-4]
        # Append the data to the list
        sampled.append({'func': content, 'idx': idx, 'target': int(target)})

# Create a DataFrame from the data
df = pd.DataFrame(sampled, columns=['func', 'idx', 'target'])

# Display the DataFrame
# print(df)

FILE_NAME="vulgen_vuls5k"
df.to_json(f'./container_data/{FILE_NAME}.jsonl',orient='records', lines=True)

In [ ]:
%ls container_data/train/ | grep '_0.c$' | wc -l
%ls container_data/train/ | grep '_1.c$' | wc -l

# Data Viz

In [24]:
from glob import glob 
# VULGEN adds 963 -> ratio = clean + x / (vul + 963) = 12024 + x / (10768 + 963) -> x = ratio * (10768 + 963) - 12024 = 1075.3 
SAMPLED = 5000*3
print(f"Items to be added to clean ones from [43]: {1.1166419019316494 * (10768 + SAMPLED) - 12024}") 

all_train_files = glob("./container_data/train/*.c")
v_train_files = glob("./container_data/train/*_1.c")
c_train_files = glob("./container_data/train/*_0.c")

print(f"all train files: {len(all_train_files)}")
print(f"all vulnerable train files: {len(v_train_files)}")
print(f"all clean train files: {len(c_train_files)}")

print(f"Train Ratio {len(c_train_files)/len(v_train_files)}") # devign clean/vul 1.1166419019316494

Items to be added to clean ones from [43]: 16749.62852897474
all train files: 27792
all vulnerable train files: 15768
all clean train files: 12024
Train Ratio 0.7625570776255708


# BigVul 

In [1]:
import pandas as pd
BIGVUL_TRAIN = "../bigvul-cleaned/clean_train.csv"


dftr = pd.read_csv(BIGVUL_TRAIN)

In [6]:
# for easier reading in RAG
dftr[['index','target', 'processed_func','func_after', 'flaw_line']].to_json(path_or_buf="./container_data/bigvul-train.jsonl",orient='records', lines=True)

In [5]:
dftr.columns.values

array(['index', 'Access Gained', 'Attack Origin',
       'Authentication Required', 'Availability', 'CVE ID', 'CVE Page',
       'CWE ID', 'Complexity', 'Confidentiality', 'Integrity',
       'Known Exploits', 'Publish Date', 'Score', 'Summary',
       'Update Date', 'Vulnerability Classification', 'add_lines',
       'codeLink', 'commit_id', 'commit_message', 'del_lines',
       'file_name', 'files_changed', 'func_after', 'func_before', 'lang',
       'lines_after', 'lines_before', 'parentID', 'patch', 'project',
       'project_after', 'project_before', 'target', 'vul_func_with_fix',
       'processed_func', 'flaw_line', 'flaw_line_index',
       'clean_processed_func'], dtype=object)

In [10]:
dftr[['index','target', 'processed_func', 'func_before', 'func_after']].head()

,index,target,processed_func,func_before,func_after
0,186765,1,BrowserContext* SharedWorkerDevToolsAgentHost:...,BrowserContext* SharedWorkerDevToolsAgentHost...,BrowserContext* SharedWorkerDevToolsAgentHost...
1,171392,0,"asocket* create_remote_socket(unsigned id, atr...","asocket* create_remote_socket(unsigned id, atr...","asocket* create_remote_socket(unsigned id, atr..."
2,71810,0,static void draw_under_color_string(DrawingWan...,static void draw_under_color_string(DrawingWan...,static void draw_under_color_string(DrawingWan...
3,141681,0,static bool isCommandLineAPIGetter(const Strin...,static bool isCommandLineAPIGetter(const Strin...,static bool isCommandLineAPIGetter(const Strin...
4,172977,0,static void rpng2_x_display_row(ulg row)\n{\n ...,static void rpng2_x_display_row(ulg row)\n{\n ...,static void rpng2_x_display_row(ulg row)\n{\n ...
